# Prepare a sci-kit grid-search / cross-val pipeline

### load data

In [2]:
import os
from google.colab import drive

drive.mount('/content/gdrive')
os.chdir('gdrive/My Drive/home/qminers')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import matplotlib.pyplot as plt
import pandas as pd
from qminers.utils import prepare_data_v1

prepare_data = prepare_data_v1

In [0]:
financial_df = pd.read_csv('data/sp500_2010_01_01-2014_07_31.csv')
calendar_df = pd.read_csv('data/econ_calendar.csv')

data_train_val, data_test = prepare_data(financial_df, calendar_df)

In [0]:
data_train_val = data_train_val.astype(float)

### prepare a custom cross-val split generator
- we want the cross-validation to be from a continuous time interval
- to make sure it doesn't overlap with training data

In [0]:
def split_generator(training_data, n_splits=5):
  n_folds = []
  fraction_size = training_data.shape[0] // n_splits
  
  for i in range(n_splits - 1):
    idx_start = i * fraction_size
    idx_end = (i + 1) * fraction_size
    
    fold = training_data.iloc[idx_start:idx_end]
    n_folds.append(fold)
   
  fold = training_data.iloc[idx_end:]
  n_folds.append(fold)

## Automated training & validation

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV


scaler, lin_regressor, transformer = None, None, None

estimators = [('preprocessing', scaler), ('model', lin_regressor)]
estimators = Pipeline(estimators)

regr = TransformedTargetRegressor(regressor=estimators,
                                   transformer=transformer)

lin_models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet()
]
svrs = [
    SVR(gamma='auto', kernel='linear'),
    SVR(gamma='auto', kernel='rbf')
]
ensembles = [
    AdaBoostRegressor(),
    AdaBoostRegressor(base_estimator=SVR(gamma='auto', kernel='linear')), # takes long
    BaggingRegressor(),
    BaggingRegressor(base_estimator=SVR(gamma='auto', kernel='linear')), # CV score=0.190 +/- 0.08
    BaggingRegressor(base_estimator=SVR(gamma='auto', kernel='rbf')), # CV score=0.168 +/- 0.06
    ExtraTreesRegressor(n_estimators=100),
    RandomForestRegressor(n_estimators=100),
    GradientBoostingRegressor(),
]

models = lin_models + svrs + ensembles
# models = ensembles

param_grid = {
    'regressor__preprocessing': [MinMaxScaler(), RobustScaler(), StandardScaler()],
    'regressor__model': models,
    'transformer': [MinMaxScaler(), RobustScaler(), StandardScaler()]
}

grid_search = GridSearchCV(
    cv=10,
    scoring=('r2', 'neg_mean_squared_error'),
    refit='r2',
    estimator=regr,
    param_grid=param_grid,
#     return_train_score=True)
    return_train_score=False)

In [83]:
grid_search.fit(data_train_val.iloc[:-1, :], data_train_val.iloc[1:, -1])

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=TransformedTargetRegressor(check_inverse=True, func=None, inverse_func=None,
              regressor=Pipeline(memory=None, steps=[('preprocessing', None), ('model', None)]),
              transformer=None),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'regressor__preprocessing': [MinMaxScaler(copy=True, feature_range=(0, 1)), RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True), StandardScaler(copy=True, with_mean=True, with_std=True)], 'regressor__model': [LinearRegression(copy_X=True, f...entering=True,
       with_scaling=True), StandardScaler(copy=True, with_mean=True, with_std=True)]},
       pre_dispatch='2*n_jobs', refit='r2', return_train_score=False,
       scoring=('r2', 'neg_mean_squared_error'), verbose=0)

In [84]:
results = pd.DataFrame(grid_search.cv_results_)
selected_columns = [x for x in results.columns if not any([y in x for y in ['param', 'split', 'time']])] 
results = results[selected_columns]
results

,mean_test_neg_mean_squared_error,mean_test_r2,rank_test_neg_mean_squared_error,rank_test_r2,std_test_neg_mean_squared_error,std_test_r2
0,-3.263012e+15,0.162149,4,27,2.065706e+15,0.244840
1,-3.263012e+15,0.162149,3,26,2.065706e+15,0.244840
2,-3.263012e+15,0.162149,2,28,2.065706e+15,0.244840
3,-3.263012e+15,0.162149,9,32,2.065706e+15,0.244840
4,-3.263012e+15,0.162149,10,34,2.065706e+15,0.244840
5,-3.263012e+15,0.162149,5,33,2.065706e+15,0.244840
6,-3.263012e+15,0.162149,6,30,2.065706e+15,0.244840
7,-3.263012e+15,0.162149,8,31,2.065706e+15,0.244840
8,-3.263012e+15,0.162149,7,29,2.065706e+15,0.244840
9,-3.395644e+15,0.172103,20,13,2.442510e+15,0.194064


In [35]:
best_row = grid_search.best_index_
print("Best parameter (CV score=%0.3f +/- %.2f):" % (results.loc[best_row]['mean_test_r2'], results.loc[best_row]['std_test_r2'] / 2))

Best parameter (CV score=0.183 +/- 0.08):


In [66]:
best_row = grid_search.best_index_
print("Best parameter (CV score=%0.3f +/- %.2f):" % (results.loc[best_row]['mean_test_r2'], results.loc[best_row]['std_test_r2'] / 2))

Best parameter (CV score=0.190 +/- 0.08):


In [85]:
best_row = grid_search.best_index_
print("Best parameter (CV score=%0.3f +/- %.2f):" % (results.loc[best_row]['mean_test_r2'], results.loc[best_row]['std_test_r2'] / 2))

Best parameter (CV score=0.187 +/- 0.09):


In [36]:
print(grid_search.best_estimator_)

TransformedTargetRegressor(check_inverse=True, func=None, inverse_func=None,
              regressor=Pipeline(memory=None,
     steps=[('preprocessing', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False))]),
              transformer=RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True))


In [86]:
print(grid_search.best_estimator_)

TransformedTargetRegressor(check_inverse=True, func=None, inverse_func=None,
              regressor=Pipeline(memory=None,
     steps=[('preprocessing', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', BaggingRegressor(base_estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, ver...mators=10, n_jobs=None, oob_score=False,
         random_state=None, verbose=0, warm_start=False))]),
              transformer=StandardScaler(copy=True, with_mean=True, with_std=True))


## Eval best estimator on test

In [0]:
from sklearn.metrics import mean_squared_error

data_test_x, data_test_y = data_test.iloc[:-1, :], data_test.iloc[1:, -1]
test_r2 = grid_search.best_estimator_.score(data_test_x, data_test_y)
print('Test R2:\t%.3f' % test_r2)

test_mse = mean_squared_error(data_test_y, grid_search.best_estimator_.predict(data_test_x))
print('Test MSE:\t%E' % test_mse)

Test R2:	-0.095
Test MSE:	1.490505E+15


In [87]:
from sklearn.metrics import mean_squared_error

data_test_x, data_test_y = data_test.iloc[:-1, :], data_test.iloc[1:, -1]
test_r2 = grid_search.best_estimator_.score(data_test_x, data_test_y)
print('Test R2:\t%.3f' % test_r2)

test_mse = mean_squared_error(data_test_y, grid_search.best_estimator_.predict(data_test_x))
print('Test MSE:\t%E' % test_mse)

Test R2:	-0.209
Test MSE:	1.646470E+15


/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
